# PYNQ tutorial: DMA to streamed interfaces

Overlay consists of two DMAs and an AXI Stream FIFO (input and output AXI stream interfaces). The FIFO represents an accelerator.

* The first DMA with read channel enabled is connected from DDR to IP input stream (reading from DDR, and sending to AXI stream).
* The second DMA has a write channel enabled and is connected to IP output stream to DDR (receiving from AXI stream, and writing to DDR memory).

There are other IP in the design which will be ignored for now.

![](data/dma_stream_example.png)

## 1. Downloading overlay
The overlay can be downloaded automatically when instantiating an overlay class.

In [ ]:
from pynq import Overlay

overlay = Overlay("./bitstream/pynq_tutorial.bit")

We can also check all the IPs in this overlay. 

In [ ]:
help(overlay)

## 2. Create DMA instances

Using the labels for the DMAs listed above, we can create two DMA objects.

In [ ]:
import pynq.lib.dma

dma_send = overlay.axi_dma_from_ps_to_pl
dma_recv = overlay.axi_dma_from_pl_to_ps

## 3. Debug DMA

In the following cell, we will create some useful methods to print control and status information from the DMA instances. Notice that read() and write() functions are available and can access the IP register space.  

In [ ]:
def print_dma_status():
    print("====  From memory to FIFO  ====")
    print("MM to Stream Control: 0x" + 
          hex(dma_send.read(0x0)))
    print("             Binary : 0b" + 
          hex(dma_send.read(0x0)))
    print("MM to Stream Status : 0x" + 
          hex(dma_send.read(0x4)))
    print("             Binary : 0b" + 
          format(dma_send.read(0x4), '0b'))
    
    print("\n==== From FIFO to Memory ====")
    print("Stream to MM Control: 0x" + 
          hex(dma_recv.read(0x30)))
    print("             Binary : 0b" + 
          hex(dma_recv.read(0x30)))
    print("Stream to MM Status : 0x" + 
          hex(dma_recv.read(0x34)))
    print("             Binary : 0b" + 
          format(dma_recv.read(0x34), '0b'))

The bits for control registers are:
* bit 1 : This bit is always 1.
* bit 16 : IRQ threshold.

The bits for status registers are:
* bit 0 : If it is 1, it means the DMA is halted.

In [ ]:
print_dma_status()

## 5. Read DMA
We will read some data from memory, and write to FIFO in the following cells.

The first step is to create the a contiguous memory block. Xlnk will be used to allocate the buffer, and NumPy will be used to specify the type of the buffer. 

In [ ]:
from pynq import Xlnk
import numpy as np

data_size = 10

xlnk = Xlnk()

input_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)

The array can be used like any other NumPy array. We can write some test data to the array. Later the data will be transferred by the DMA to the FIFO. 

In [ ]:
for i in range(data_size):
    input_buffer[i] = i + 0xffff0000

Let's check the contents of the array. The data in the following cell will be sent from PS (DDR memory) to PL (streaming FIFO).

In [ ]:
for i in range(data_size):
    print(hex(input_buffer[i]))

Now we are ready to carry out DMA transfer from a memory block in DDR to FIFO.

In [ ]:
dma_send.sendchannel.transfer(input_buffer)

Let's check the status of the DMA.

Control register:
* bit 0 : Start the DMA if the value is 1, else halt the DMA.
* bit 1 : Always 1
* bit 16 : IRQ threshold.

Status register:
* bit 0 : 0 means the DMA is halted.
* bit 1 : 1 means the DMA is idle.
* bit 12 : Interrupt on complete.

In [ ]:
print_dma_status()

## 5. Write DMA
Let's read the data back from FIFO stream, and write to MM memory. The steps are similar.

We will prepare an empty array before reading data back from FIFO.

In [ ]:
output_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)
for i in range(data_size):
    print('0x' + format(output_buffer[i], '02x'))

In [ ]:
dma_recv.recvchannel.transfer(output_buffer)

The next cell will print out the data received from PL (streaming FIFO) to PS (DDR memory). This should be the same as the data we sent previously.

In [ ]:
for i in range(data_size):
    print('0x' + format(output_buffer[i], '02x'))

In [ ]:
print_dma_status()

## 6. Free all the memory buffers
Don't forget to free all the memory buffers to avoid memory leaks!

In [ ]:
xlnk.xlnk_reset()